In [ ]:
import spacy

In [ ]:
nlp = spacy.load('en')

# **Training the Entity Recognizer**

## **Deciding Whether You Need to Train the Entity Recognizer**

In [ ]:
doc = nlp(u'Coult you pick me at Solnce?')

In [ ]:
for ent in doc.ents:
  print(ent.text, ent.label_)

Solnce ORG


In [ ]:
print(spacy.explain('ORG'))

Companies, agencies, institutions, etc.


## **Creating Training Examples**

In [ ]:
train_exams = [
    ('Could you send a taxi to Solnce?', {'entities': [(25, 32, 'GPE')]}),
    ('Is there a flat rate to the airport from Solnce?', {'entities': [(41, 48, 'GPE')]}),
    ('How long is the wait for a taxi right now?', {'entities': []})
]

In [ ]:
First = 'Could you send a taxi to Solnce?'

In [ ]:
len(First)

32

In [ ]:
First[25:32]

'Solnce?'

In [ ]:
Second = 'Is there a flat rate to the airport from Solnce?'

In [ ]:
len(Second)

48

In [ ]:
Second[41:48]

'Solnce?'

## **Automating the example creation process**

In [ ]:
doc = nlp(u'Could you send a taxi to Solnce? I need to get to Google. Could you send a taxi an hour later?')

 **If we pick up the content from file**

In [ ]:
# f = open("test.txt", "rb")
# contents = f.read()
# doc = nlp(contents.decode('utf8'))

In [ ]:
train_exams = []
districts = ['Solnce', 'Greenwal', 'Downtown']
for sent in doc.sents:
  entities = []
  for token in sent:
    if token.ent_type != 0:
      start = token.idx - sent.start_char
      if token.text in districts:
        entity = (start, start + len(token), 'GPE')
      else:
        entity = (start, start + len(token), token.ent_type_)
      entities.append(entity)

  tpl = (sent.text, {'entities': entities})

  train_exams.append(tpl)


In [ ]:
train_exams

[('Could you send a taxi to Solnce?', {'entities': [(25, 31, 'GPE')]}),
 ('I need to get to Google.', {'entities': [(17, 23, 'ORG')]}),
 ('Could you send a taxi an hour later?', {'entities': []})]

## **Disabling the Other Pipeline Components**

In [ ]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe !='ner']
nlp.disable_pipes(*other_pipes)

[('tagger', <spacy.pipeline.pipes.Tagger at 0x7fb031748fd0>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x7fb0316226a8>)]

## **The Training Process**

In [ ]:
import random
from spacy.util import minibatch, compounding

In [ ]:
optimizer = nlp.entity.create_optimizer()

In [ ]:
for i in range(25):
  random.shuffle(train_exams)
  max_batch_size = 3
  batch_size = compounding(2.0, max_batch_size, 1.001)
  batches = minibatch(train_exams, size=batch_size)
  for batch in batches:
    texts, annotations = zip(*batch)
    nlp.update(texts, annotations, sgd=optimizer)

  ner = nlp.get_pipe('ner')
  ner.to_disk('usr/to/ner')

## **Evaluating the Updated Recognizer**

In [ ]:
from spacy.pipeline import EntityRecognizer

In [ ]:
nlp = spacy.load('en', disable=['ner'])

In [ ]:
ner = EntityRecognizer(nlp.vocab)

In [ ]:
ner.from_disk('/content/usr/to/ner')

In [ ]:
nlp.add_pipe(ner, "custom_ner")

In [ ]:
print(nlp.meta['pipeline'])

['tagger', 'parser', 'custom_ner']


In [ ]:
doc = nlp(u'Could you pick me up at Solnce?')

In [ ]:
for ent in doc.ents:
  print(ent.text, ent.label_)

Solnce GPE


# **Creating a New Dependency Parser**

## **Custom Synctactic Parsing to Understand User Input**

In [ ]:
doc = nlp(u'Find a high paid job with no experience.')

In [ ]:
print([(t.text, t.dep_, t.head.text) for t in doc])

[('Find', 'ROOT', 'Find'), ('a', 'det', 'job'), ('high', 'advmod', 'paid'), ('paid', 'amod', 'job'), ('job', 'dobj', 'Find'), ('with', 'prep', 'Find'), ('no', 'det', 'experience'), ('experience', 'pobj', 'with'), ('.', 'punct', 'Find')]


## **Deciding on Types of Semantic Relations to Use**

## **Creating Training Examples**

In [ ]:
TRAINING_DATA = [
    ('find a high paying job with no experience', {
        'heads': [0, 4, 4, 4, 0, 7, 7, 4],
        'deps' : ['ROOT', '-', 'QUALITY', 'QUALITY', 'ACTIVITY', '-', 'QUALITY', 'ATTRIBUTE']
    }),

    ('find good workout classes near home', {
        'heads': [0, 4, 4, 4, 0, 4],
        'deps' : ['ROOT', '-', 'QUALITY', 'ACTIVITY', 'QUALITY', 'ATTRIBUTE']
    })


]

In [ ]:
doc = nlp(u'find a high paying job with no experience')
heads = []
for token in doc:
    heads.append(token.head.i)

print(heads)

[0, 4, 4, 4, 0, 0, 7, 5]


## **Training the Parser**

In [ ]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe !='ner']
nlp.disable_pipes(*other_pipes)

[('parser', <spacy.pipeline.pipes.DependencyParser at 0x7fb02f8a9048>)]

In [ ]:
parser = nlp.create_pipe('parser')
nlp.add_pipe(parser, first=True)
for text, annotations in TRAINING_DATA:
    for d in annotations.get('deps', []):
        parser.add_label(d)

optimizer = nlp.begin_training()
import random
for i in range(25):
    random.shuffle(TRAINING_DATA)
    for text, annotations in TRAINING_DATA:
        nlp.update([text], [annotations], sgd=optimizer)

parser.to_disk('/content/oracle/to/parser')

In [ ]:
test = []
for text, annotations in TRAINING_DATA:
  print(text, annotations)
  for d in annotations.get('deps', []):
    print(d)


find a high paying job with no experience {'heads': [0, 4, 4, 4, 0, 7, 7, 4], 'deps': ['ROOT', '-', 'QUALITY', 'QUALITY', 'ACTIVITY', '-', 'QUALITY', 'ATTRIBUTE']}
ROOT
-
QUALITY
QUALITY
ACTIVITY
-
QUALITY
ATTRIBUTE
find good workout classes near home {'heads': [0, 4, 4, 4, 0, 4], 'deps': ['ROOT', '-', 'QUALITY', 'ACTIVITY', 'QUALITY', 'ATTRIBUTE']}
ROOT
-
QUALITY
ACTIVITY
QUALITY
ATTRIBUTE
